In [35]:
import tensorflow as tf
import numpy as np
import cv2

In [36]:
import glob
import os

In [40]:
def loadTrainImages(trainPath, imageSize, classes = ["dogs", "cats"]):
    """Load images for training given the path to the folder that contains
    the images. Return the images and its labels.
    
    trainPath: path to folder that contains images to train
    imageSize: the size used in cv2.resize()
    """
    images = []
    labels = []

    for className in classes:   
        index = classes.index(className)
        path = os.path.join(trainPath, className, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (imageSize, imageSize),0,0, cv2.INTER_LINEAR)
            image = image.astype(np.float32)
            image = np.multiply(image, 1.0 / 255.0)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [41]:
np.random.seed(1)
tf.set_random_seed(1)

In [42]:
trainPath = "train"
imageSize = 64
images, labels = loadTrainImages(trainPath, imageSize)

In [46]:
classes = ["dogs", "cats"]
numOfChannels = 3
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, imageSize,imageSize,numOfChannels], name='x')
trueY = tf.placeholder(tf.float32, shape=[None, len(classes)], name='trueY')

In [48]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [49]:
def create_convolutional_layer(input,
               numOfChannels, 
               sizeOfFilter,        
               numOfFilters):
    
    """Returns a constructed convolutional layer.
    
    numOfChannels: number of channels. For RGB it's 3.
    sizeOfFilter: size of filter used in convolutional layer.
    numOfFilters: number of filters (feature maps used in the convolutional lyaer)
    """
    # randomly initialize weights and bias to train for this layer
    weights = create_weights(shape=[sizeOfFilter, sizeOfFilter, numOfChannels, numOfFilters])
    biases = create_biases(numOfFilters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    # max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    # activating with relu
    layer = tf.nn.relu(layer)

    return layer

In [57]:
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])

    return layer

In [51]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [47]:
filterSize = 3
# num of filters in convolution layer 1,2,3
numOfLayersConv1 = 16
numOfLayersConv2 = 16
numOfLayersConv3 = 32
# size of fully connected layer
sizeOfFCLayer = 64

In [55]:
layer_conv1 = create_convolutional_layer(x,
               numOfChannels,
               filterSize,
               numOfLayersConv1)
layer_conv2 = create_convolutional_layer(layer_conv1,
               numOfLayersConv1,
               filterSize,
               numOfLayersConv2)

layer_conv3= create_convolutional_layer(layer_conv2,
               numOfLayersConv2,
               filterSize,
               numOfLayersConv3)

In [58]:
layer_flat = create_flatten_layer(layer_conv3)

In [60]:
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=sizeOfFCLayer,
                     use_relu=True)

In [61]:
layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=sizeOfFCLayer,
                     num_outputs=len(classes),
                     use_relu=False) 

In [64]:
y_pred = tf.nn.softmax(layer_fc2,name='y_pred')
y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=trueY)

In [65]:
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

NameError: name 'y_true_cls' is not defined